In [1]:
import pandas as pd
import numpy as np
import os
import pdfplumber as pdfp
import tabula
import re
import PyPDF2
import requests
import datetime
from datetime import datetime

In [2]:
# diretório
diretorio = r'C:\Users\Computadores Gamer\OneDrive\Área de Trabalho\contas_ape'
diretorio = diretorio.replace('\\','/')
os.chdir(diretorio)

print(os.getcwd())

C:\Users\Computadores Gamer\OneDrive\Área de Trabalho\contas_ape


In [3]:
# arquivos
arquivos = os.listdir()
print(arquivos)

['boleto_condominio_0823.pdf', 'boleto_enel_0823.pdf', 'boleto_quintoandar_0823.pdf', 'contas_apartamento.ipynb']


In [19]:
# parametros para donwload do arquivos (mês e ano)
ano = '23'
mes = '8'

# selecao dos arquivos de acordo com os parâmetros
arquivos_corretos = []
for i in arquivos:
    if i[:-4].endswith(mes + ano):
        arquivos_corretos.append(i)

print(arquivos_corretos)


['boleto_condominio_0823.pdf', 'boleto_enel_0823.pdf', 'boleto_quintoandar_0823.pdf']


In [20]:
# download dos arquivos
dic = {}

for indice, i in enumerate(arquivos_corretos):
    nome_variavel = f'{i}'
    dic[nome_variavel] = pdfp.open(i)
dic


{'boleto_condominio_0823.pdf': <pdfplumber.pdf.PDF at 0x296dfea7610>,
 'boleto_enel_0823.pdf': <pdfplumber.pdf.PDF at 0x296e01dbe50>,
 'boleto_quintoandar_0823.pdf': <pdfplumber.pdf.PDF at 0x296e01da950>}

In [ ]:
# ler determinada conta. As paginas ja estao pré determinadas, visto que é sempre o mesmo documento, a única coisa que muda é o valor

# paginas
# boleto_condominio, pagina 0
# boleto_enel, pagina 1
# boleto_quintoandar, pagina 0

# extrair texto
# boleto_condominio, 
# boleto_enel, pagina 1
# boleto_quintoandar, pagina 0



In [65]:
# paginas a serem lidas - condominio
boleto_condominio.pages
pagina_condominio = boleto_condominio.pages[0]
print(pagina_condominio)

<Page:1>


In [66]:
# extraindo texto - boleto condominio
texto_condominio = pagina_condominio.extract_text().split('\n')
texto_condominio


['Recibo do Pagador',
 'Sispac - Sistema Padrao De Administracao De Condominios',
 'Ltda Vencimento 10/09/2023',
 'Rua Constantino de Sousa, 1372 - Campo Belo',
 'Valor Total R$ 1566.00',
 'Cep: 04605003 - São Paulo - SP',
 'Fone: (11) 5561-0464 Nr. Recibo 0799482',
 'Site: https://cysne.com.br/ - E-mail: contato@sispac.com.br',
 'AROSA',
 'Condômino: PHILIPPE ANTOINE BEKMESSIAN E OUTRO Unidade: 061',
 'Agência/Código Beneficiário Nosso Número (-) Desconto/Abatimento (+) Mora/Multa (=) Valor Cobrado',
 '0066/26867-3 109-00799482-2',
 'CONDOMINIO SET/2023 R$ 1.380,00',
 'F RESERVA SET/2023 R$ 74,00',
 '13o SALARIO /FÉRIAS SET/2023 R$ 97,00',
 'RESCISAO SET/2023 R$ 10,00',
 'DESPESAS CORREIO R$ 5,00',
 'Esta quitação só terá validade após o pagamento do cheque pelo banco sacado. Autenticação',
 'Mecânica Não serão aceitos pagamentos por DOC ou Depósito Bancário. Recibo do Pagador Caso',
 'Autenticação Mecânica Recibo do Pagador',
 'os mesmos sejam feitos não valerão como quitação para es

In [67]:
# linhas a serem lidas
for i in texto_condominio:
    if 'CONDOMINIO' in i:
        print (i)
        indice = texto_condominio.index(i)
        print(indice)

CONDOMINIO SET/2023 R$ 1.380,00
12


In [68]:
# obtencao do valor_condominio
valor_condominio = texto_condominio[indice].split()[3] # como há um indice só nao preciso movimentar esse indice
valor_condominio = valor_condominio.replace('.','')
valor_condominio = valor_condominio.replace(',','.')
valor_condominio = float(valor_condominio)

print(pagina_condominio)

<Page:1>


In [69]:
for i in texto_condominio:
    if 'Vencimento' in i:
        print (i)
        indice = texto_condominio.index(i)
        print(indice)

Ltda Vencimento 10/09/2023
2
Local do Pagamento Vencimento
23
Após o Vencimento Multa de ........... R$31,32 ( - ) Outras Deduções
36


In [70]:
vencimento_condominio = texto_condominio[indice-34].split()[2] # o ultimo indice é o 36, portanto preciso subir 34 linhas pra chegar no valor da fatura do indice 2
vencimento_condominio = datetime.strptime(vencimento_condominio, '%d/%m/%Y')
vencimento_condominio = vencimento_condominio.date() # arrancar o horário

print(vencimento_condominio)

2023-09-10


In [71]:
# paginas - boleto enel
boleto_enel.pages
pagina_enel = boleto_enel.pages[1]
print(pagina_enel)

<Page:2>


In [72]:
# extraindo texto - boleto enel
texto_enel = pagina_enel.extract_text().split('\n')
texto_enel

['2',
 '- Segunda via de fatura',
 '25 AGO 2023 AGO 2023 12 SET 2023',
 'PREFEITURA DO MUNICÍPIO 0800 77 90 156',
 '- CLIENTE, PAGUE PREFERENCIALMENTE NOS CANAIS',
 'ELETRÔNICOS',
 '- CADASTRE SUA FATURA EM DÉBITO AUTOMÁTICO',
 'UTILIZANDO O NUMERO DA CONTA CONFORME EXIBIDO',
 'AO LADO',
 '- CONTRIBUA COM O MEIO AMBIENTE, SEJA DIGITAL E',
 'SOLICITE O RECEBIMENTO DE SUA FATURA POR E-MAIL E',
 'ECONOMIZE PAPEL. ACESSE NOSSO',
 '502916928898 25 AGO 2023 AGO 2023',
 '52170381 12 SET 2023 202,62',
 'MATEUS RODRIGUES DE SOUZA',
 '- ENCARGOS POR ATRASO SERÃO COBRADOS NA PRÓXIMA FATURA',
 '- APROVEITE OS BENEFÍCIOS DO DÉBITO AUTOMÁTICO',
 '836100000022 026200481005 073999787317 002192878284']

In [73]:
# verificando onde esta o valor
for i in texto_enel:
    if 'BENEFÍCIOS ' in i:
        print(i)
        indice_enel = texto_enel.index(i)
        print(indice_enel)
    

    

- APROVEITE OS BENEFÍCIOS DO DÉBITO AUTOMÁTICO
16


In [74]:
# chegando na linha do valor da fatura e extraindo o valor
valor_enel = texto_enel[indice_enel-3].split(' ')
valor_enel = valor_enel[4].replace(',','.')
valor_enel = float(valor_enel)

In [75]:
# data de vencimento - boleto enel
vencimento_enel = texto_enel[indice_enel-3].split(' ')


In [76]:
# paginas - quinto andar
boleto_quintoandar.pages
pagina_quintoandar = boleto_quintoandar.pages[0]
pagina_quintoandar

<Page:1>

In [77]:
# extraindo texto - boleto quinto andar
texto_quintoandar = pagina_quintoandar.extract_text().split('\n')
texto_quintoandar

['Recibo do Pagador',
 '341-7 34191.09073 33218.790674 03922.160001 2 94670000311391',
 'Pagador/CPF/CNPJ/Endereço',
 'Mateus Rodrigues de Souza CPF/CNPJ: 067.390.056-80',
 'Rua Graúna, 134 61, Vila Uberabinha, CEP: 04514-000 - São Paulo-SP',
 'Pagador/Avalista:',
 'Nosso Número Nº do Documento Data de Vencimento Valor do Documento (=) Valor Cobrado',
 '109/07332187-9-9 465616B308 08/09/2023 3113,91',
 'Nome do Beneficiário/CPF/CNPJ/Endereço',
 'QUINTO ANDAR SERVICOS IMOBILIARIOS LTDA CPF/CNPJ: 16.788.643/0001-81',
 'Agência / Código do Beneficiário Autenticação Mecânica',
 '0670/39221-6',
 '341-7 34191.09073 33218.790674 03922.160001 2 94670000311391',
 'Local de Pagamento Data de Vencimento',
 'PAGÁVEL EM QUALQUER BANCO MESMO APÓS O VENCIMENTO 08/09/2023',
 'Beneficiário/CPF/CNPJ Agência / Código do Beneficiário',
 'QUINTO ANDAR SERVICOS IMOBILIARIOS LTDA CPF/CNPJ: 16.788.643/0001-81 0670/39221-6',
 'Data do Documento Nr. do Documento Espécie Doc. Aceite Data de Processamento Nosso N

In [78]:
# verifica onde esta o valor - boleto quinto andar
for i in texto_quintoandar:
    if 'R$' in i:
        print (i)
        indice_quintoandar = texto_quintoandar.index(i)
        print(indice_quintoandar)

109 R$ 3113,91
20
APOS VENCIMENTO COBRAR MULTA R$ 311,39 E JUROS 1,04 AO DIA (+) Mora / Multa
23


In [79]:
# linha do valor boleto quinto andar
valor_quintoandar = texto_quintoandar[indice_quintoandar-3].split(' ')
valor_quintoandar = float(valor_quintoandar[2].replace(',','.'))
valor_quintoandar

3113.91

In [80]:
# tabela contas
# valor_condominio, valor_enel, valor_quintoandar
soma_contas = valor_condominio+valor_enel+valor_quintoandar

contas = {'condominio':[valor_condominio],
          'energia':[valor_enel],
          'quinto andar':[valor_quintoandar],
          'soma':[soma_contas]}

tabela_contas = pd.DataFrame(contas)
tabela_contas




,condominio,energia,quinto andar,soma
0,1380.0,202.62,3113.91,4696.53
